## Load data

In [1]:
# see more data/movie-actor/make_casting_graph.py

import sys
import pickle
from pprint import pprint 


# pre defined casting weight graph
with open('data/movie-actor/casting_graph.pkl', 'rb') as f:
    graph = pickle.load(f)

# create idx to actor name function
with open('data/movie-actor/actors.csv', encoding='utf-8') as f:
    next(f)
    docs = [line[:-1].split('\t') for line in f]
    # English name if exist else Korean name
    _idx2actor = {doc[0]:(doc[2] if doc[2] else doc[1]) for doc in docs}

# create idx to movie name function
def append_year_countries(year, countries):
    if year and countries:
        return ' ({}, {})'.format(year, countries)
    elif year:
        return ' ({})'.format(year)
    elif countries:
        return ' ({})'.format(countries)
    return ''

with open('data/movie-actor/movies.csv', encoding='utf-8') as f:
    next(f)
    docs = [line[:-1].split('\t') for line in f]
    _idx2movie = {doc[0]:'{}{}'.format(doc[1], append_year_countries(doc[4], doc[5])) for doc in docs if len(docs)}
    _idx2year = {doc[0]:(int(doc[4]) if doc[4] else 0) for doc in docs if len(docs)}

# create idx to num comments
with open('data/movie-actor/num_comments.txt', encoding='utf-8') as f:
    docs = [line[:-1].split('\t') for line in f]
    _idx2numcomments = {movie_idx:int(num) for movie_idx, num in docs}

idx2movie = lambda idx: _idx2movie.get(idx, 'Unknown')
idx2year = lambda idx: _idx2year.get(idx, 0)
idx2actor = lambda idx: _idx2actor.get(idx, 'Unknown')
idx2numcomments = lambda idx: _idx2numcomments.get(idx,0)


# Transform one-way to bidirected
sys.path.append('data/movie-actor/')
from make_casting_graph import oneway_to_bidirected_graph

g = oneway_to_bidirected_graph(graph)

## Implementation with Python dict

In [2]:
list(g.keys())[:5]

['movie 22682', 'actor 7995', 'actor 4009', 'movie 25882', 'actor 154441']

In [3]:
# making bias
bias = {node:(idx2numcomments(node.split()[1]) if node[0] == 'm' else 0) for node in g}
_sum = sum(bias.values())
bias = {node:b / _sum for node, b in bias.items()}

### Train PageRank

In [4]:
import time
from pagerank import pagerank

t = time.time()
rank = pagerank(g,
                bias=bias,
                df=0.15,
                max_iter=30,
                converge_error=0.001,
                verbose=1)

t = time.time() - t
print('computation time : %.3f sec' % t)

Iteration = 1, diff = 0.9735227399459792, sum = 0.9999999999998657
Iteration = 2, diff = 0.7586092188109216, sum = 0.9999999999989707
Iteration = 3, diff = 0.612136395804903, sum = 1.0000000000001668
Iteration = 4, diff = 0.5041426449280965, sum = 0.9999999999996744
Iteration = 5, diff = 0.4205911434661837, sum = 1.0000000000000329
Iteration = 6, diff = 0.35282857845349685, sum = 0.9999999999997901
Iteration = 7, diff = 0.29719895865328017, sum = 1.0000000000000615
Iteration = 8, diff = 0.25081504214011324, sum = 1.0000000000002625
Iteration = 9, diff = 0.2120067786422557, sum = 0.9999999999999264
Iteration = 10, diff = 0.17937385549497817, sum = 0.999999999999767
Iteration = 11, diff = 0.15187850974117678, sum = 1.0000000000001241
Iteration = 12, diff = 0.1286725921232049, sum = 1.000000000000105
Iteration = 13, diff = 0.1090574423391369, sum = 1.0000000000001614
Iteration = 14, diff = 0.09246572410873297, sum = 1.000000000000169
Iteration = 15, diff = 0.07842074920691962, sum = 0.999

### Check whether famous movie has high rank

In [6]:
movie_rank = {node:rank for node, rank in rank.items() if node[0] == 'm'}
actor_rank = {node:rank for node, rank in rank.items() if node[0] == 'a'}

for movie, _ in sorted(movie_rank.items(), key=lambda x:-x[1])[:100]:
    movie_idx = movie.split()[1]
    print(idx2movie(movie_idx))

인천상륙작전 (2016, 한국)
연평해전 (2015, 한국)
국가대표 (2009, 한국)
설국열차 (2013, 한국)
어벤져스: 에이지 오브 울트론 (2015, 미국)
국제시장 (2014, 한국)
겨울왕국 (2014, 미국)
암살 (2015, 한국)
7번방의 선물 (2013, 한국)
캡틴 아메리카: 시빌 워 (2016, 미국)
곡성(哭聲) (2016, 한국)
귀향 (2016, 한국)
26년 (2012, 한국)
인터스텔라 (2014, 미국 영국)
세 얼간이 (2011, 인도)
부산행 (2016, 한국)
아가씨 (2016, 한국)
레미제라블 (2012, 영국)
밀정 (2016, 한국)
럭키 (2016, 한국)
디 워 (2007, 한국)
판도라 (2016, 한국)
미스터 고 (2013, 한국)
다세포 소녀 (2006, 한국)
해운대 (2009, 한국)
님아, 그 강을 건너지 마오 (2014, 한국)
광해, 왕이 된 남자 (2012, 한국)
수상한 그녀 (2014, 한국)
트랜스포머: 사라진 시대 (2014, 미국)
킹스맨 : 시크릿 에이전트 (2015, 미국 영국)
괴물 (2006, 한국)
아수라 (2016, 한국)
역린 (2014, 한국)
포화 속으로 (2010, 한국)
어거스트 러쉬 (2007, 미국)
비긴 어게인 (2014, 미국)
검은 사제들 (2015, 한국)
아바타 (2009, 미국)
말할 수 없는 비밀 (2008, 대만)
베테랑 (2015, 한국)
덕혜옹주 (2016, 한국)
감기 (2013, 한국)
타워 (2012, 한국)
라스트 갓파더 (2010, 한국 미국)
늑대소년 (2012, 한국)
왕의 남자 (2005, 한국)
그래비티 (2013, 미국 영국)
영웅: 샐러멘더의 비밀 (2013, 한국 미국 러시아 연방)
라디오 스타 (2006, 한국)
과속스캔들 (2008, 한국)
신비한 동물사전 (2016, 미국 영국)
쎄시봉 (2015, 한국)
미녀는 괴로워 (2006, 한국)
퍼시픽 림 (2013, 미국)
다이빙벨 (2014, 한국)
클레멘타인 (200

## Implementation with numpy

In [7]:
import numpy as np
from scipy.sparse import csc_matrix

# node index
nodes = set(g.keys())
idx2node = list(sorted(nodes))
node2idx = {node:idx for idx, node in enumerate(idx2node)}

# bias
bias = np.asarray([b for node, b in sorted(bias.items(), key=lambda tp:node2idx[tp[0]])])
print(bias.shape)

# transform g to sparse matrix
rows = []
cols = []
data = []

for from_node, to_dict in g.items():
    from_idx = node2idx[from_node]
    for to_node, weight in to_dict.items():
        to_idx = node2idx[to_node]
        rows.append(from_idx)
        cols.append(to_idx)
        data.append(weight)

A = csc_matrix((data, (rows, cols)))
print(A.shape)

(265607,)
(265607, 265607)


### Train PageRank

In [8]:
from sklearn.preprocessing import normalize

t = time.time()

max_iter = 30
df = 0.85

ir = 1 / A.shape[0]
rank = np.asarray([ir] * A.shape[0])

for n_iter in range(1, max_iter + 1):
    rank_new = A.dot(rank) # call scipy.sparse safe_sparse_dot()
    rank_new = normalize(rank_new.reshape(1, -1), norm='l1').reshape(-1)
    rank_new = df * rank_new + (1 - df) * bias    
    diff = abs(rank - rank_new).sum()
    rank = rank_new
    print('iter {} : diff = {}'.format(n_iter, diff))

t = time.time() - t
print(t)

iter 1 : diff = 0.281436840863022
iter 2 : diff = 0.3970505378527365
iter 3 : diff = 0.17598404082440902
iter 4 : diff = 0.14807658516192676
iter 5 : diff = 0.09962315062320268
iter 6 : diff = 0.0765602968138843
iter 7 : diff = 0.058114626092355424
iter 8 : diff = 0.042949603335787775
iter 9 : diff = 0.03410703651803252
iter 10 : diff = 0.024663500399523496
iter 11 : diff = 0.020053770053409213
iter 12 : diff = 0.014331538003882077
iter 13 : diff = 0.011801592719125686
iter 14 : diff = 0.008385166921666137
iter 15 : diff = 0.0069544954062276815
iter 16 : diff = 0.004925645467125705
iter 17 : diff = 0.004106103285551124
iter 18 : diff = 0.002901720075500885
iter 19 : diff = 0.002430137343265774
iter 20 : diff = 0.0017143036073690432
iter 21 : diff = 0.001442295384774437
iter 22 : diff = 0.0010158693712554695
iter 23 : diff = 0.0008586564597530771
iter 24 : diff = 0.0006037068986667392
iter 25 : diff = 0.0005127185595059035
iter 26 : diff = 0.00035977812698531186
iter 27 : diff = 0.00030

### Check whether famous movie has high rank

In [9]:
rank_ = {idx2node[idx]:value for idx, value in enumerate(rank)}
movierank = {node:value for node, value in rank_.items() if 'movie' in node}
actorrank = {node:value for node, value in rank_.items() if 'actor' in node}

for movie, value in sorted(movierank.items(), key=lambda x:-x[1])[:50]:
    movie_idx = movie.split()[1]
    print(idx2movie(movie_idx))

26년 (2012, 한국)
부산행 (2016, 한국)
디 워 (2007, 한국)
곡성(哭聲) (2016, 한국)
7번방의 선물 (2013, 한국)
인터스텔라 (2014, 미국 영국)
인천상륙작전 (2016, 한국)
국제시장 (2014, 한국)
괴물 (2006, 한국)
국가대표 (2009, 한국)
암살 (2015, 한국)
베테랑 (2015, 한국)
아바타 (2009, 미국)
연평해전 (2015, 한국)
설국열차 (2013, 한국)
말할 수 없는 비밀 (2008, 대만)
겨울왕국 (2014, 미국)
왕의 남자 (2005, 한국)
캡틴 아메리카: 시빌 워 (2016, 미국)
님아, 그 강을 건너지 마오 (2014, 한국)
늑대소년 (2012, 한국)
귀향 (2016, 한국)
과속스캔들 (2008, 한국)
어벤져스: 에이지 오브 울트론 (2015, 미국)
세 얼간이 (2011, 인도)
다세포 소녀 (2006, 한국)
검사외전 (2016, 한국)
아저씨 (2010, 한국)
군도:민란의 시대 (2014, 한국)
광해, 왕이 된 남자 (2012, 한국)
해적: 바다로 간 산적 (2014, 한국)
해운대 (2009, 한국)
터널 (2016, 한국)
화려한 휴가 (2007, 한국)
아가씨 (2016, 한국)
럭키 (2016, 한국)
다크 나이트 라이즈 (2012, 미국 영국)
다이빙벨 (2014, 한국)
덕혜옹주 (2016, 한국)
아수라 (2016, 한국)
다크 나이트 (2008, 미국)
밀정 (2016, 한국)
인셉션 (2010, 미국 영국)
포화 속으로 (2010, 한국)
전우치 (2009, 한국)
검은 사제들 (2015, 한국)
히말라야 (2015, 한국)
트랜스포머 (2007, 미국)
7광구 (2011, 한국)
좋은 놈, 나쁜 놈, 이상한 놈 (2008, 한국)
